# Ruta 2 (Baldwin-lite) - Entrenamiento en Colab (GPU L4)

Este notebook ejecuta `entrenar_pipeline_completo.py` usando GPU (idealmente L4).

## Requisitos
- Tener el repo en GitHub **o** subir la carpeta `cDAQ_9174` a Google Drive.
- En Colab: `Runtime -> Change runtime type -> GPU`.


In [ ]:
!nvidia-smi
import torch
print('torch', torch.__version__)
print('cuda available', torch.cuda.is_available())
if torch.cuda.is_available():
    print('device', torch.cuda.get_device_name(0))


## Opción A: clonar repo (recomendado)
Edita `REPO_URL` con tu URL (HTTPS) y ejecuta.


In [ ]:
# @title Clone repo (EDITA REPO_URL)
REPO_URL = ''  # e.g. https://github.com/USUARIO/REPO.git
BRANCH = ''    # opcional, e.g. 'main'

if not REPO_URL:
    raise ValueError('Set REPO_URL first')

!rm -rf cDAQ_9174
if BRANCH:
    !git clone --depth 1 -b {BRANCH} {REPO_URL} cDAQ_9174
else:
    !git clone --depth 1 {REPO_URL} cDAQ_9174

%cd cDAQ_9174
!ls


## Opción B: usar Google Drive (si no quieres clonar)
Sube la carpeta `cDAQ_9174` a tu Drive, luego ajusta `DRIVE_PROJECT_DIR`.


In [ ]:
# @title Mount Drive (opcional)
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PROJECT_DIR = ''  # e.g. /content/drive/MyDrive/cDAQ_9174
if DRIVE_PROJECT_DIR:
    %cd $DRIVE_PROJECT_DIR
    !ls


In [ ]:
# Instalar dependencias mínimas
!python -m pip install -q tqdm pandas matplotlib


In [ ]:
# Entrenar (maximizando GPU L4)
# Ajusta épocas/batch/AMP vía env vars (soportado por scripts/entrenar_ruta2.py)

import os

# Objetivo: saturar GPU sin OOM
# - AMP reduce memoria y acelera
# - num_workers acelera carga (en Colab suele ir bien 2-8)
# - batch size: si OOM, baja a 512/256

os.environ["USE_AMP"] = "1"
os.environ["NUM_WORKERS"] = "4"

# Más épocas para generalización multi-sesión (Baldwin-lite)
os.environ["EPOCHS_STAGE1"] = "1200"
os.environ["EPOCHS_STAGE2"] = "2400"

# Empieza agresivo, baja si truena
os.environ["BATCH_SIZE"] = "1024"

# Ejecutar script desde la raíz del repo (asumiendo que estamos en cDAQ_9174/ o root del repo)
try:
    !python scripts/entrenar_ruta2.py
except Exception as e:
    print('Fallo con BATCH_SIZE=1024, intentando 512...')
    os.environ["BATCH_SIZE"] = "512"
    !python scripts/entrenar_ruta2.py


## Salidas
Al terminar, quedan en `levitador valentin/`:
- `flux_observer_final.pt`
- `position_predictor_final.pt`
- `sensorless_pipeline_final.pt`

Si trabajas desde Drive, ya quedan persistidos.
